In [ ]:
import threading
import time
import random

class CountingSemaphore:
    def __init__(self, initial):
        self.value = initial
        self.lock = threading.Lock()
        self.condition = threading.Condition(self.lock)

    def wait(self):
        with self.lock:
            self.value -= 1
            if self.value < 0:
                # Block the thread
                self.condition.wait()

    def signal(self):
        with self.lock:
            self.value += 1
            if self.value <= 0:
                # Wake one blocked thread
                self.condition.notify()

In [ ]:
BUFFER_SIZE = 5

empty = CountingSemaphore(BUFFER_SIZE)  # empty slots - Counting semaphore
full = CountingSemaphore(0)              # filled slots - Counting semaphore
mutex = CountingSemaphore(1)             # binary semaphore (mutex)

buffer = []


In [ ]:
def camera():
    for i in range(10):
        time.sleep(random.uniform(0.4, 1.0))

        empty.wait()
        mutex.wait()

        buffer.append(f"Frame-{i}")
        print(f"[Camera] Produced Frame-{i} | Buffer size: {len(buffer)}")

        mutex.signal()
        full.signal()

In [ ]:
def satellite():
    for _ in range(10):
        full.wait()
        mutex.wait()

        frame = buffer.pop(0)
        print(f"    [Satellite] Consumed {frame} | Buffer size: {len(buffer)}")

        mutex.signal()
        empty.signal()

        time.sleep(random.uniform(0.6, 1.2))

In [ ]:
producer = threading.Thread(target=camera)
consumer = threading.Thread(target=satellite)

producer.start()
consumer.start()

producer.join()
consumer.join()

print("\nSimulation finished.")

[Camera] Produced Frame-0 | Buffer size: 1
    [Satellite] Consumed Frame-0 | Buffer size: 0
[Camera] Produced Frame-1 | Buffer size: 1
    [Satellite] Consumed Frame-1 | Buffer size: 0
[Camera] Produced Frame-2 | Buffer size: 1
[Camera] Produced Frame-3 | Buffer size: 2
    [Satellite] Consumed Frame-2 | Buffer size: 1
[Camera] Produced Frame-4 | Buffer size: 2
    [Satellite] Consumed Frame-3 | Buffer size: 1
[Camera] Produced Frame-5 | Buffer size: 2
[Camera] Produced Frame-6 | Buffer size: 3
    [Satellite] Consumed Frame-4 | Buffer size: 2
[Camera] Produced Frame-7 | Buffer size: 3
[Camera] Produced Frame-8 | Buffer size: 4
    [Satellite] Consumed Frame-5 | Buffer size: 3
[Camera] Produced Frame-9 | Buffer size: 4
    [Satellite] Consumed Frame-6 | Buffer size: 3
    [Satellite] Consumed Frame-7 | Buffer size: 2
    [Satellite] Consumed Frame-8 | Buffer size: 1
    [Satellite] Consumed Frame-9 | Buffer size: 0

Simulation finished.
